In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5*5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d (1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d (6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120)
  (fc2): Linear(in_features=120, out_features=84)
  (fc3): Linear(in_features=84, out_features=10)
)


In [2]:
params = list(net.parameters())
print (len(params))
for i in range(len(params)):
    print (params[i].size())

10
torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 400])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


In [3]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print (out)

Variable containing:
 0.0848 -0.0198  0.0001  0.0417 -0.0839  0.0558  0.1080 -0.0610 -0.1482 -0.0983
[torch.FloatTensor of size 1x10]



In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [5]:
output = net (input)
target = Variable(torch.arange(1, 11))
criterion = nn.MSELoss()

loss = criterion(output, target)
print (loss)

Variable containing:
 38.8907
[torch.FloatTensor of size 1]



In [6]:
print (loss.grad_fn)
print (loss.grad_fn.next_functions[0][0])
print (loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [7]:
net.zero_grad()

print('conv1.bias.grad before backward')
print (net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
  6.8478
 -3.6338
  2.6551
  4.1743
  3.8209
 -5.6541
[torch.FloatTensor of size 6]



In [8]:
# stochastic gradient descent method
# new_val = val - lr * grad, net.parameters() is an set of data which have to update
learning_rate = 0.01
for f in net.parameters():
    f.data = f.data - (f.grad.data * learning_rate)

In [12]:
import torch.optim as optim
optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.zero_grad()
output = net(input)
target = Variable(torch.arange(1, 11))
criterion = nn.MSELoss()
loss = criterion(output, target)
loss.backward()
optimizer.step()